In [1]:
import numpy as np
from sklearn.decomposition import pca
import time
from sklearn import metrics
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.optimizers import RMSprop

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## 读取MNIST数据
* 两组数据
* 1. train 1-2000, test 2001-4000
* 2. train 2001-4000, test 1-2000
* 数据是按0-9的顺序排列的，每个数字有两百个

In [2]:
def load_data(flag=1,normalize=False):
    """
    flag: 选择两组数据中的某一个
    normalize: 数据是否要正则化 (在训练某些model的时候好像有bug，待研究)
    
    return: X_train, Y_train, X_test, Y_test
    """
    labels = get_data('digits4000_txt/digits4000_digits_labels.txt')
    digits = get_data('digits4000_txt/digits4000_digits_vec.txt')

    if flag==1:
        X_train = digits[:2000]
        Y_train = labels[:2000]
        X_test = digits[2000:]
        Y_test = labels[2000:]
    else:
        X_train = digits[2000:]
        Y_train = labels[2000:]
        X_test = digits[:2000]
        Y_test = labels[:2000]

    if normalize:
        X_train = X_train/255
        Y_train = Y_train/255
        X_test = X_test/255
        Y_test = Y_test/255

    return np.array(X_train),np.array(Y_train),np.array(X_test),np.array(Y_test)

def get_data(filename):
    return np.loadtxt(filename)

In [3]:
x_train, y_train, x_test, y_test = load_data(flag=1,normalize=False)

## 各种Classifier的实现

### KNN Classifier

In [3]:
def knn_classifier(train_x,train_y):
    from sklearn.neighbors import KNeighborsClassifier
    model = KNeighborsClassifier()
    model.fit(train_x, train_y)
    return model

### Logistic Classifier

In [4]:
def logistic_regression_classifier(train_x, train_y):
    from sklearn.linear_model import LogisticRegression
    model = LogisticRegression(penalty='l2')
    model.fit(train_x, train_y)
    return model

### Decision Tree Classifier

In [5]:
def decision_tree_classifier(train_x, train_y):
    from sklearn import tree
    model = tree.DecisionTreeClassifier()
    model.fit(train_x, train_y)
    return model

### SVM Classifier

In [6]:
def svm_classifier(train_x, train_y):
    from sklearn.svm import SVC
    model = SVC(kernel='rbf', gamma=0.01, C=1) # 选择kernel之后，rbf的准确率比其他的高，参数还没研究过
    model.fit(train_x, train_y)
    return model

### CNN model
* 模型参数直接在这个里面改
* CNN不需要进行PCA

In [7]:
def cnn_classifier(train_x, train_y):
    train_X = train_x.reshape(-1, 1, 28, 28)
    train_Y = np_utils.to_categorical(train_y, num_classes=10)
    
    model = Sequential()
    
    model.add(Convolution2D(filters=32,
                                        kernel_size=5,
                                        strides=1,
                                        padding='same',
                                        batch_input_shape=(None, 1, 28, 28),
                                        data_format='channels_first'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=2, 
                                       strides=2, 
                                       padding='same', 
                                       data_format='channels_first'))
    model.add(Convolution2D(64, 5, 
                                        strides=1, 
                                        padding='same', 
                                        data_format='channels_first'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(2, 2, 'same', data_format='channels_first'))
    
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    adam = Adam(lr=1e-4)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(train_X, train_Y, epochs=20, batch_size=64)
    
    return model
    

### DNN model
* 和CNN model相似，结构不一样
* DNN可以进行PCA

In [8]:
def dnn_classifier(train_x, train_y):
    batch_size = 100
    nb_classes = 10
    nb_epoch = 20
    
    train_y = np_utils.to_categorical(train_y, num_classes=10)
    
    input_dim = train_x.shape[1]
    
    model = Sequential()
    model.add(Dense(512, input_shape=(input_dim,)))
    
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(512))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
    model.fit(train_x, train_y, batch_size=batch_size, epochs=nb_epoch)
    
    return model

## 进行分析

### 使用PCA降维

### PCA model

In [9]:
def PCA_model(train_x,components=600):
    model = pca.PCA(n_components=components).fit(train_x)
    return model

### training和testing的部分

In [10]:
# classifiers = ['knn_classifier','logistic_regression_classifier','decision_tree_classifier','svm_classifier','cnn_classifier','dnn_classifier']
classifiers = ['knn_classifier','logistic_regression_classifier','decision_tree_classifier','svm_classifier','dnn_classifier'] # test
x_train, y_train, x_test, y_test = load_data(flag=1,normalize=False)

In [11]:
def training_process(classifiers, x_train, y_train, x_test, y_test):
    result = {}
    for classifier in classifiers:
        try:
            print("=======================")
            print('Classifier: {}'.format(classifier))
            temp_model = eval(classifier)(x_train,y_train) 
    
            if classifier == 'cnn_classifier': # CNN需要转换一下数据格式
                x_test_reshape = x_test.reshape(-1, 1, 28, 28)
                y_test_reshape = np_utils.to_categorical(y_test, num_classes=10)
                loss, accuracy = temp_model.evaluate(x_test_reshape, y_test_reshape)
            else:
                if classifier == 'dnn_classifier':
                    y_test_reshape = np_utils.to_categorical(y_test, num_classes=10)
                    loss, accuracy = temp_model.evaluate(x_test, y_test_reshape)
                else:
                    y_train_predict = temp_model.predict(x_train)
                    training_accuracy = metrics.accuracy_score(y_train,y_train_predict)
                    print('training accuracy: {}'.format("%.2f%%"%( 100*training_accuracy)))
                    y_predict = temp_model.predict(x_test)
                    accuracy = metrics.accuracy_score(y_test, y_predict)
                
            print('testing accuracy: {}'.format("%.2f%%"%( 100*accuracy)))
            result[classifier] = accuracy
        except:
            print('+++++++++++++++++++++++++')
            print('Error with {}.'.format(classifier))
            import traceback
            print(traceback.format_exc())
            print('+++++++++++++++++++++++++')
    return result

### 没有normalization和PCA

In [12]:
training_process(classifiers, x_train, y_train, x_test, y_test)

Classifier: knn_classifier
training accuracy: 94.50%
testing accuracy: 91.70%
Classifier: logistic_regression_classifier
training accuracy: 100.00%
testing accuracy: 83.45%
Classifier: decision_tree_classifier
training accuracy: 100.00%
testing accuracy: 73.50%
Classifier: svm_classifier
training accuracy: 100.00%
testing accuracy: 90.10%
Classifier: dnn_classifier
Epoch 1/20
2000/2000 [==============================] - 1s 363us/step - loss: 14.2963 - acc: 0.1115
Epoch 2/20
2000/2000 [==============================] - 0s 197us/step - loss: 14.4902 - acc: 0.1010
Epoch 3/20
2000/2000 [==============================] - 0s 199us/step - loss: 14.4902 - acc: 0.1010
Epoch 4/20
2000/2000 [==============================] - 0s 195us/step - loss: 14.4982 - acc: 0.1005
Epoch 5/20
2000/2000 [==============================] - 0s 144us/step - loss: 13.9985 - acc: 0.1310
Epoch 6/20
2000/2000 [==============================] - 0s 138us/step - loss: 13.9383 - acc: 0.1345
Epoch 7/20
2000/2000 [==========

{'knn_classifier': 0.917,
 'logistic_regression_classifier': 0.8345,
 'decision_tree_classifier': 0.735,
 'svm_classifier': 0.901,
 'dnn_classifier': 0.257}

### 有normalization，没有PCA

In [13]:
training_process(classifiers, x_train/255, y_train, x_test/255, y_test)

Classifier: knn_classifier
training accuracy: 94.50%
testing accuracy: 91.70%
Classifier: logistic_regression_classifier
training accuracy: 98.60%
testing accuracy: 88.50%
Classifier: decision_tree_classifier
training accuracy: 100.00%
testing accuracy: 72.25%
Classifier: svm_classifier
training accuracy: 100.00%
testing accuracy: 90.10%
Classifier: dnn_classifier
Epoch 1/20
2000/2000 [==============================] - 1s 281us/step - loss: 1.0501 - acc: 0.6635
Epoch 2/20
2000/2000 [==============================] - 0s 162us/step - loss: 0.4182 - acc: 0.8690
Epoch 3/20
2000/2000 [==============================] - 0s 149us/step - loss: 0.2935 - acc: 0.9080
Epoch 4/20
2000/2000 [==============================] - 1s 263us/step - loss: 0.2058 - acc: 0.9335
Epoch 5/20
2000/2000 [==============================] - 0s 205us/step - loss: 0.1513 - acc: 0.9575
Epoch 6/20
2000/2000 [==============================] - 0s 146us/step - loss: 0.1072 - acc: 0.9670
Epoch 7/20
2000/2000 [=================

{'knn_classifier': 0.917,
 'logistic_regression_classifier': 0.885,
 'decision_tree_classifier': 0.7225,
 'svm_classifier': 0.901,
 'dnn_classifier': 0.937}

### 没有normalization，有PCA
* CNN不适用于PCA

### 训练PCA model

In [14]:
pca_model = PCA_model(x_train)

In [16]:
x_train_pca = pca_model.transform(x_train)
x_test_pca = pca_model.transform(x_test)
temp_classifiers = ['knn_classifier','logistic_regression_classifier','decision_tree_classifier','svm_classifier','dnn_classifier'] # 没有CNN
training_process(temp_classifiers, x_train_pca, y_train, x_test_pca, y_test)

Classifier: knn_classifier
training accuracy: 94.50%
testing accuracy: 91.70%
Classifier: logistic_regression_classifier
training accuracy: 99.85%
testing accuracy: 75.95%
Classifier: decision_tree_classifier
training accuracy: 100.00%
testing accuracy: 64.45%
Classifier: svm_classifier
training accuracy: 100.00%
testing accuracy: 90.10%
Classifier: dnn_classifier
Epoch 1/20
2000/2000 [==============================] - 1s 285us/step - loss: 13.0490 - acc: 0.1900
Epoch 2/20
2000/2000 [==============================] - 0s 128us/step - loss: 11.0509 - acc: 0.3090
Epoch 3/20
2000/2000 [==============================] - 0s 127us/step - loss: 10.2640 - acc: 0.3560
Epoch 4/20
2000/2000 [==============================] - 0s 135us/step - loss: 9.4031 - acc: 0.4105
Epoch 5/20
2000/2000 [==============================] - 0s 126us/step - loss: 8.8793 - acc: 0.4425
Epoch 6/20
2000/2000 [==============================] - 0s 127us/step - loss: 8.6636 - acc: 0.4580
Epoch 7/20
2000/2000 [==============

{'knn_classifier': 0.917,
 'logistic_regression_classifier': 0.7595,
 'decision_tree_classifier': 0.6445,
 'svm_classifier': 0.901,
 'dnn_classifier': 0.514}

### 有normalization和PCA

In [17]:
pca_model = PCA_model(x_train/255,components=400)
x_train_normal_pca = pca_model.transform(x_train/255)
x_test_normal_pca = pca_model.transform(x_test/255)
temp_classifiers = ['knn_classifier','logistic_regression_classifier','decision_tree_classifier','svm_classifier','dnn_classifier'] # 没有CNN
training_process(temp_classifiers, x_train_normal_pca, y_train, x_test_normal_pca, y_test)

Classifier: knn_classifier
training accuracy: 94.55%
testing accuracy: 91.70%
Classifier: logistic_regression_classifier
training accuracy: 98.35%
testing accuracy: 87.90%
Classifier: decision_tree_classifier
training accuracy: 100.00%
testing accuracy: 64.40%
Classifier: svm_classifier
training accuracy: 100.00%
testing accuracy: 90.10%
Classifier: dnn_classifier
Epoch 1/20
2000/2000 [==============================] - 1s 277us/step - loss: 0.9646 - acc: 0.7305
Epoch 2/20
2000/2000 [==============================] - 0s 107us/step - loss: 0.3099 - acc: 0.9205
Epoch 3/20
2000/2000 [==============================] - 0s 105us/step - loss: 0.1685 - acc: 0.9590
Epoch 4/20
2000/2000 [==============================] - 0s 107us/step - loss: 0.1037 - acc: 0.9740
Epoch 5/20
2000/2000 [==============================] - 0s 106us/step - loss: 0.0497 - acc: 0.9905
Epoch 6/20
2000/2000 [==============================] - 0s 106us/step - loss: 0.0270 - acc: 0.9970
Epoch 7/20
2000/2000 [=================

{'knn_classifier': 0.917,
 'logistic_regression_classifier': 0.879,
 'decision_tree_classifier': 0.644,
 'svm_classifier': 0.901,
 'dnn_classifier': 0.9115}